## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os, traceback
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
print(os.environ["FABRIC_PROJECT_ID"])

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

Force orchestrator to get list of available resources afresh from the broker by passing `force_refresh=True`. Otherwise, cached model is returned.

In [ ]:
status, advertised_topology = slice_manager.resources(force_refresh=True)

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
status, advertised_topology = slice_manager.resources(force_refresh=True, level=2)

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
for s in advertised_topology.sites.values():
    print(f"{s.name} [Site]")
    print(f"     Maitenance State: {s.maintenance_info}")
    if s.capacities is None:
        continue
    print(f"     cpu: {s.capacities.cpu}/{s.capacity_allocations.cpu if s.capacity_allocations is not None else 0}, "
              f"core: {s.capacities.core}/{s.capacity_allocations.core if s.capacity_allocations is not None else 0}, "
              f"ram: {s.capacities.ram}/{s.capacity_allocations.ram if s.capacity_allocations is not None else 0}, "
              f"disk: {s.capacities.disk}/{s.capacity_allocations.disk if s.capacity_allocations is not None else 0}",
              f"unit: {s.capacities.unit}/{s.capacity_allocations.unit if s.capacity_allocations is not None else 0}")
    
    ##### Get Nodes from Site ##### Hack until FIM fix available
    from fim.graph.abc_property_graph import ABCPropertyGraph
    from fim.view_only_dict import ViewOnlyDict
    from fim.user import Node
    node_id_list = s.topo.graph_model.get_first_neighbor(node_id=s.node_id, rel=ABCPropertyGraph.REL_HAS,
                                                         node_label=ABCPropertyGraph.CLASS_NetworkNode)
    ret = dict()
    for nid in node_id_list:
        _, node_props = s.topo.graph_model.get_node_properties(node_id=nid)
        n = Node(name=node_props[ABCPropertyGraph.PROP_NAME], node_id=nid, topo=s.topo)
        # exclude Facility nodes
        from fim.user import NodeType
        if n.type != NodeType.Facility:
            ret[n.name] = n
    nodes = ViewOnlyDict(ret)
    ##### Get Nodes from Site ##### Hack until FIM fix available
    
    
    print(f"     Workers:")    
    for w in nodes.values():
        print(f"          {w.name}")
        print(f"               cpu: {w.capacities.cpu}/{w.capacity_allocations.cpu if w.capacity_allocations is not None else 0}, "
              f"core: {w.capacities.core}/{w.capacity_allocations.core if w.capacity_allocations is not None else 0}, "
              f"ram: {w.capacities.ram}/{w.capacity_allocations.ram if w.capacity_allocations is not None else 0}, "
              f"disk: {w.capacities.disk}/{w.capacity_allocations.disk if w.capacity_allocations is not None else 0}")
        print(f"               Components:")
        for c in w.components.values():
            print(f"                    {c.name}"
                  f" unit: {c.capacities.unit}/{c.capacity_allocations.unit if c.capacity_allocations is not None else 0}")
            if str(c.type) == 'NVME':
                  print(f"                               disk: {c.capacities.disk}/{c.capacity_allocations.disk if c.capacity_allocations is not None else 0}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()